In [ ]:
from renderer import Chat, DB

In [ ]:
chat = Chat.fromKey("914472")

In [ ]:
doc = chat.build()

In [ ]:
with open("test.md", "w") as f:
    f.writelines(doc.render())

In [ ]:
from collections.abc import Iterable
def func():
    yield "foo"

it = (i for i in range(10))
isinstance(it, Iterable)

In [ ]:
from utils import flatten

list(flatten([
    [1,2,3],
    [1,2,3],
    [1,2,3],
    [1,2,3],
    [
        [1],
        [1],
        [1],
        [1],
    ],
    map(
        lambda x:x+1,
        [1,2,3,4]
    )
]))

In [ ]:
import json
from renderer import DB
with open("foobar.json", "w") as f:
    json.dump(DB.getDocument("chat-logs", "891409"), f)

In [ ]:
from utils import Matcher, matched_filter, buffered

In [ ]:
from markdown import Document, Blockquote, BlockquoteTag, Text, Details, CodeBlock, Wrapper

txt = Text(
    Text.Text("Hello World"), Text.Linebreak(),
    Text.Code("Some Code"), Text.Text(" and some more text")
)

item = BlockquoteTag(
    txt,
    BlockquoteTag(
        txt,
        Details(
            txt,
            CodeBlock(["import foo", "foo.bar()", "print('wow')"], lang="python"),
            summary="MyDetails",
        )
    )
)

doc = Document(
    Wrapper(
        item,
        item
    )
)

In [ ]:
with open("test.md", "w") as f:
    f.writelines(doc.render())

In [ ]:
mybool = True

In [ ]:
mybool ^= True

In [ ]:
mybool

In [ ]:

[1, *(1 if True else ())]